# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### 1. What test is appropriate for this problem? Does CLT apply?

A two-sample z-test is used to test the difference between two populations and I believe would be a good fit for this data. Yes, the Central Limit Theorem applies becuase the observations in both samples are independent from each other and were randomly selected, so we can assume they are represenative of the population:

$$z = \frac{\left( \hat{p}_W - \hat{p}_B \right) - 0}{\sqrt{\hat{p} (1 - \hat{p)} \left( \frac{1}{n_W} + \frac{1}{n_B}\right)}}$$

where

$$\hat{p}_W = \frac{x_W}{n_W }$$
$$\hat{p}_B = \frac{x_B}{n_B }$$

$$\hat{p} = \frac{x_W + x_B}{n_W + n_B}$$

The null and alternate hypotheses:

$$H_0: p_B = p_W$$
$$H_A: p_B \ne p_W$$

In [ ]:
w_name = data[data['race'] =='w']
b_name = data[data['race'] =='b']


prop_w = np.sum(w_name['call']) / len(w_name)
prop_b = np.sum(b_name['call']) / len(b_name)

prop_diff = prop_w - prop_b
p_hat = (np.sum(w_name['call']) + np.sum(b_name['call'])) / (len(w_name) + len(b_name))

z = prop_diff / np.sqrt(p_hat * (1 - p_hat) * ((1 / len(w_name)) + (1 / len(b_name))))
pval = stats.norm.cdf(-z) * 2
print("Z score: {}".format(z))
print("P-value: {}".format(pval))

$ MoE = Z * \sqrt{\hat{p} (1 - \hat{p)} \left( \frac{1}{n_W} + \frac{1}{n_B}\right)}$

$ CI = \left(\hat{p}_W - \hat{p}_B\right)  +_- MoE $

In [15]:
MoE = 1.96 * np.sqrt(p_hat*(1-p_hat)*((1/len(w_name))+(1/len(b_name))))
CI = prop_diff + np.array([-1,1]) * MoE
print('Margin of Error: {}'.format(MoE))
print('Confidence Interval: {}'.format(CI))

Margin of Error: 0.015281912310894095
Confidence Interval: [ 0.01675094  0.04731477]



With the p-value being so small, we can reject the null hypothesis that people with white sounding names and people with black sounding names recieve the same callback rate. 

With the calculated margin of error and confidence interval

## Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Our analysis does not conclude that race/name is the most important factor in callvack success, because we have not compliled and compared every other possible variable. We can conclude that there is a significant difference between race/name and callback success.